In [ ]:
%%capture
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import timeit

import delfi.distribution as dd
import delfi.inference as infer
import delfi.generator as dg

from delfi.simulator import GaussMixture
import delfi.summarystats as ds
from delfi.utils.viz import plot_pdf

# problem setup

In [ ]:
seed=42
return_abs = False

# analytic target posterior for plotting (y-axis flipped for overlay with imshow figures !)
p_true = dd.MoG(a=[0.5, 0.5], 
                ms=[np.asarray([.5, -.5]), np.asarray([-.5, .5])], 
                Ss=[0.01*np.eye(2), 0.01*np.eye(2)])
p_true.ndim=2


# very basic approach to controlling generator seeds
def init_g(seed):
    m = GaussMixture(dim=2, bimodal=True, return_abs=return_abs, noise_cov=[0.01, 0.01], seed=seed)
    p = dd.Uniform(lower=[-1,-1], upper=[1,1], seed=seed)
    s = ds.Identity()
    return dg.Default(model=m, prior=p, summary=s)

g = init_g(seed=seed)

obs_stats = np.array([[.5, -.5]])

trn_data = g.gen(1000)
plt.figure(figsize=(6,5))
plt.plot(trn_data[1], trn_data[0], '.')
plt.xlabel('x')
plt.ylabel('theta(1) resp. theta(2)')
plt.title('marginals p(theta_i, x)')
plt.show()
        
# training schedule
n_train=500
n_rounds=10

# fitting setup
minibatch=100
epochs=[500//(r+1) for r in range(n_rounds)]

# network setup
n_hiddens=[50,50]
reg_lambda=0.01

# convenience
pilot_samples=0
svi=False
verbose=True
prior_norm=False
init_norm=False

# SNPE-C parameters
n_null = 1

# MAF parameters
mode='random' # ordering of variables for MADEs
n_mades = 5 # number of MADES
act_fun = 'tanh'
batch_norm = False # batch-normalization currently not supported
train_on_all = False

# MDN parameters
n_components = 2


# discrete-proposal SNPE-C (MAF)

In [ ]:
# control MAF seed
rng = np.random
rng.seed(seed)

# generator
g = init_g(seed=seed)

# inference object
res = infer.SNPEC(g, 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 seed=seed, 
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 n_mades=n_mades, # providing this argument triggers usage of MAFs (vs. MDNs)
                 act_fun=act_fun,
                 mode=mode, 
                 rng=rng, 
                 batch_norm=batch_norm, 
                 verbose=verbose,
                 prior_norm=prior_norm)

# train

t = timeit.time.time()

logs, tds, posteriors = res.run(n_train=n_train, 
                    proposal='discrete',
                    moo='q_phi_x',
                    n_null = n_null,
                    n_rounds=n_rounds,
                    train_on_all=train_on_all,
                    minibatch=minibatch, 
                    epochs=epochs)

print(timeit.time.time() -  t)


In [ ]:
from delfi.utils.viz import probs2contours

# plot loss curves
plt.figure(figsize=(12, 8))
for r in range(n_rounds):
    plt.subplot(np.ceil(n_rounds/3), 3, r+1)
    plt.plot(logs[r]['loss'])
    plt.title('loss for round r=' + str(r+1))
plt.show()


# plot posterior estimates (overlaid with ground-truth)
xo = 1.*obs_stats.flatten()
lims = np.array([[-1,1], [-1,1]])
i,j,resolution = 0,1,100
xx = np.linspace(lims[i, 0], lims[i, 1], resolution)
yy = np.linspace(lims[j, 0], lims[j, 1], resolution)
X, Y = np.meshgrid(xx, yy)
xy = np.concatenate(
    [X.reshape([-1, 1]), Y.reshape([-1, 1])], axis=1)
plt.figure(figsize=(16, 16))
plt.subplot(np.ceil(n_rounds/3), 3, 1)
pp = g.prior.eval(xy, log=False).reshape(list(X.shape))
plt.imshow(0*pp.T, origin='lower',
               extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
               aspect='auto', interpolation='none')
pp = p_true.eval(xy, log=False).reshape(list(X.shape))
plt.contour(Y, X, probs2contours(pp, levels=(0.68, 0.95)), levels=(0.68, 0.95), colors=('w', 'y'))
plt.title('prior (real posterior overlaid)')
for r in range(n_rounds):
    plt.subplot(np.ceil(n_rounds/3), 3, r + 2)
    posterior = posteriors[r] 
    pp = posterior.eval(xy, log=False).reshape(list(X.shape))
    plt.imshow(pp.T, origin='lower',
               extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
               aspect='auto', interpolation='none')
    pp = p_true.eval(xy, log=False).reshape(list(X.shape))
    plt.contour(Y, X, probs2contours(pp, levels=(0.68, 0.95)), levels=(0.68, 0.95), colors=('w', 'y'))
    plt.title('posterior estimate after round r='+str(r+1))
    #plt.savefig('/home/mackelab/Desktop/SNPE_10rounds_posterior_est.pdf')

plt.show()

In [ ]:
# control MAF seed
rng = np.random
rng.seed(seed)

# generator
g = init_g(seed=seed)

# inference object
res = infer.SNPEC(g, 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 seed=seed, 
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 n_mades=n_mades, # providing this argument triggers usage of MAFs (vs. MDNs)
                 act_fun=act_fun,
                 mode=mode, 
                 rng=rng, 
                 batch_norm=batch_norm, 
                 verbose=verbose,
                 prior_norm=prior_norm)

# train

t = timeit.time.time()

logs, tds, posteriors = res.run(n_train=n_train, 
                    proposal='discrete',
                    moo='p_tilda',
                    n_null = n_null,
                    n_rounds=n_rounds,
                    train_on_all=train_on_all,
                    minibatch=minibatch, 
                    epochs=epochs)

print(timeit.time.time() -  t)


In [ ]:
from delfi.utils.viz import probs2contours

# plot loss curves
plt.figure(figsize=(12, 8))
for r in range(n_rounds):
    plt.subplot(np.ceil(n_rounds/3), 3, r+1)
    plt.plot(logs[r]['loss'])
    plt.title('loss for round r=' + str(r+1))
plt.show()


# plot posterior estimates (overlaid with ground-truth)
xo = 1.*obs_stats.flatten()
lims = np.array([[-1,1], [-1,1]])
i,j,resolution = 0,1,100
xx = np.linspace(lims[i, 0], lims[i, 1], resolution)
yy = np.linspace(lims[j, 0], lims[j, 1], resolution)
X, Y = np.meshgrid(xx, yy)
xy = np.concatenate(
    [X.reshape([-1, 1]), Y.reshape([-1, 1])], axis=1)
plt.figure(figsize=(16, 16))
plt.subplot(np.ceil(n_rounds/3), 3, 1)
pp = g.prior.eval(xy, log=False).reshape(list(X.shape))
plt.imshow(0*pp.T, origin='lower',
               extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
               aspect='auto', interpolation='none')
pp = p_true.eval(xy, log=False).reshape(list(X.shape))
plt.contour(Y, X, probs2contours(pp, levels=(0.68, 0.95)), levels=(0.68, 0.95), colors=('w', 'y'))
plt.title('prior (real posterior overlaid)')
for r in range(n_rounds):
    plt.subplot(np.ceil(n_rounds/3), 3, r + 2)
    posterior = posteriors[r] 
    pp = posterior.eval(xy, log=False).reshape(list(X.shape))
    plt.imshow(pp.T, origin='lower',
               extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
               aspect='auto', interpolation='none')
    pp = p_true.eval(xy, log=False).reshape(list(X.shape))
    plt.contour(Y, X, probs2contours(pp, levels=(0.68, 0.95)), levels=(0.68, 0.95), colors=('w', 'y'))
    plt.title('posterior estimate after round r='+str(r+1))
    #plt.savefig('/home/mackelab/Desktop/SNPE_10rounds_posterior_est.pdf')

plt.show()

# SNL (MAF)

In [ ]:
import sys
import snl.inference.nde as nde
from snl.ml.models.mafs import ConditionalMaskedAutoregressiveFlow
from delfi.utils.delfi2snl import SNLprior, SNLmodel


# control MAF seed
rng = np.random
rng.seed(seed)

# explicit call to MAF constructor
theta, x = g.gen(1)
n_inputs, n_outputs  = x.size, theta.size
model = ConditionalMaskedAutoregressiveFlow(
                n_inputs=n_inputs,
                n_outputs=n_outputs,
                n_hiddens=n_hiddens,
                act_fun=act_fun,
                n_mades=n_mades,
                mode=mode,
                rng=rng)


# generator
g = init_g(seed=seed)

# inference object
inf = nde.SequentialNeuralLikelihood(SNLprior(g.prior),               # method to draw parameters  
                                     SNLmodel(g.model, g.summary).gen # method to draw summary stats
                                    )

# train
t = timeit.time.time()

rng = np.random # control  
rng.seed(seed)  # MCMC seed
model = inf.learn_likelihood(obs_stats.flatten(), model, n_samples=n_train, n_rounds=n_rounds, 
                             train_on_all=True, thin=10, save_models=False, 
                             logger=sys.stdout, rng=rng)

print(timeit.time.time() -  t)


### visualize learned likelihood

In [ ]:
import snl.inference.mcmc as mcmc
log_posterior = lambda t: model.eval([t, obs_stats.flatten()]) + inf.prior.eval(t)
sampler = mcmc.SliceSampler(x=inf.all_ps[-1][-1], lp_f=log_posterior, thin=100)
ps = sampler.gen(1000)

In [ ]:
xo = 1.*obs_stats.flatten()

lims = np.array([[-1,1], [-1,1]])
i,j,resolution = 0,1, 100
        
xx = np.linspace(lims[i, 0], lims[i, 1], resolution)
yy = np.linspace(lims[j, 0], lims[j, 1], resolution)
X, Y = np.meshgrid(xx, yy)
xy = np.concatenate(
    [X.reshape([-1, 1]), Y.reshape([-1, 1])], axis=1)
pp = model.eval((xo, xy), log=False).reshape(list(X.shape))

plt.imshow(pp.T, origin='lower',
           extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
           aspect='auto', interpolation='none')
#plt.savefig('/home/mackelab/Desktop/SNL_5rounds_final_likelihood_tightll.pdf')
plt.show()

In [ ]:
plt.figure(figsize=(16,11))
for r in range(n_rounds):
    plt.subplot(np.ceil(n_rounds/3), 3, r + 1)
    plt.plot(inf.all_ps[r][:,0],
             inf.all_ps[r][:,1], 'k.')
    plt.axis([-1,1,-1,1])
    plt.xlabel('theta1')
    plt.xlabel('theta2')
    plt.title('round r='+str(r))

plt.subplot(np.ceil(n_rounds/3), 3, n_rounds+1)
plt.plot(ps[:,0],
         ps[:,1], 'k.')
plt.axis([-1,1,-1,1])
plt.xlabel('theta1')
plt.xlabel('theta2')
plt.title('round r='+str(n_rounds))
#plt.savefig('/home/mackelab/Desktop/posteriors_per_round_tightll.pdf')
    
plt.show()

# SNPE A (always MDN)

In [ ]:
# generator
g = init_g(seed=seed)

# inference object
res_A = infer.CDELFI(g, 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 n_components=n_components,
                 seed=seed, 
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 verbose=verbose,
                 init_norm=init_norm,
                 prior_norm=prior_norm)

# train
t = timeit.time.time()

logs_A, tds_A, posteriors_A = res_A.run(n_train=n_train, 
                    n_rounds=n_rounds, 
                    minibatch=minibatch, 
                    epochs=np.max(epochs))

print(timeit.time.time() -  t)


# discrete-proposal SNPE C (MDN)

In [ ]:
# generator
g = init_g(seed=seed)

# inference object
res_C = infer.SNPEC(g, 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 n_components=n_components,
                 seed=seed, 
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 verbose=verbose,
                 prior_norm=prior_norm)

# train
t = timeit.time.time()

logs_C, tds_C, posteriors_C = res_C.run(n_train=n_train, 
                    proposal='discrete',
                    moo='resample',
                    n_null = n_null,
                    n_rounds=n_rounds, 
                    minibatch=minibatch, 
                    epochs=epochs)

print(timeit.time.time() -  t)


# Gaussian-proposal SNPE-C (MDN)

In [ ]:
# generator
g = init_g(seed=seed)

# inference object
res_gC = infer.SNPEC(g, 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 n_components=n_components,
                 seed=seed, 
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 verbose=verbose,
                 prior_norm=prior_norm)

# train
t = timeit.time.time()

logs_gC, tds_gC, posteriors_gC = res_gC.run(n_train=n_train, 
                    proposal='gaussian',
                    n_rounds=n_rounds, 
                    minibatch=minibatch, 
                    epochs=epochs)

print(timeit.time.time() -  t)


## direct comparison SNPE-C against SNPE-A (both MDN)

In [ ]:
for r in range(n_rounds):
    fig,_ = plot_pdf(posteriors_A[r],
             pdf2=posteriors_C[r],
             lims=[[-1,1],[-1,1]],
             ticks=True,             
             #pdf2=g.prior,
             #gt=np.array([5,-5]), 
             resolution=100,
             figsize=(16,16));
    fig.suptitle('final posterior estimate vs MCMC samples and prior', fontsize=14)
    fig.show()
